In [0]:
from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image
import os
import sys
import threading

class Node:
    def __init__(self, item=None, link=None):
        self.item = item
        self.link = link

class CircleLinkedList:
    def __init__(self):
        self.root = Node()
        self.curNode = self.root

    def append(self, item):
        newNode = Node(item)
        curNode = self.root
        if curNode.item == None:
            self.root = newNode
            newNode.link = self.root
        else:
            while curNode.link != self.root:
                curNode = curNode.link
            curNode.link = newNode
            newNode.link = self.root

    def print(self):
        curNode = self.root
        print(curNode.item)
        while curNode.link != self.root:
            curNode = curNode.link
            print(curNode.item)

    def listSize(self):
        curNode = self.root
        cnt = 1
        while curNode.link != self.root:
            curNode = curNode.link
            cnt += 1
        return cnt

    def insert(self, idx, item):
        n = self.listSize()
        if idx < 0 or idx >= n:
            print("index range error")
        else:
            newNode = Node(item)
            curNode = self.root
            if idx == 0:
                _tmp = self.root
                self.root = newNode
                newNode.link = _tmp
            else:
                for curIdx in range(idx - 1):
                    curNode = curNode.link
                _tmp = curNode.link
                curNode.link = newNode
                newNode.link = _tmp

    def setCurrent(self, item):
        self.curNode = self.find(item)[1]

    def moveNext(self):
        self.curNode = self.curNode.link

    def delete(self, item):
        delYN = False
        curNode = self.root
        while curNode.link != self.root:
            curNode = curNode.link
        endNode = curNode
        curNode = self.root
        preNode = curNode
        if curNode.item == item:
            self.root = self.root.link
            endNode.link = self.root
            delYN = True
        else:
            while curNode.link != self.root:
                preNode = curNode
                curNode = curNode.link
                if curNode.item == item:
                    preNode.link = curNode.link
                    delYN = True
        if curNode.item == item:
            preNode.link = self.root
            delYN = True
        if delYN == False:
            print("delete failed")

class Window:
    def __init__(self):
        self.window=tk.Tk()
        self.window.title("Image Viewer")
        self.window.geometry("640x480")
        self.window.resizable(False, False)

        menubar = tk.Menu(self.window)
        menu_1 = tk.Menu(menubar, tearoff=0)
        menu_2 = tk.Menu(menubar, tearoff=0)
        menu_1.bind('<<MenuSelect>>')
        menu_2.bind('<<MenuSelect>>')

        menu_1.add_command(label="Open", command=self.dirSelect)
        menu_1.add_command(label="Close", command=self.close)
        menu_2.add_command(label="next", command=self.next)
        menubar.add_cascade(label="File", menu=menu_1)
        menubar.add_cascade(label="Image", menu=menu_2)

        self.window.config(menu=menubar)

        self.imgLabel = tk.Label(self.window, width=400, height=400, relief='solid')
        self.imgLabel.pack()
        self.window.mainloop()

    def close(self):
        self.window.quit()
        self.window.destroy()

    def dirSelect(self):
        global pngLists
        selDir = filedialog.askdirectory(iniitialdir = "/", title = "Select directory")
        print(selDir)
        pngfiles = [x for x in os.listdir(selDir) if x.endswith(".jpg")]
        print(pngfiles)

        pngLists = CircleLinkedList()
        for fname in pngfiles:
            pngLists.append(selDir + "/" + fname)
        pngLists.print()
        selFile = pngLists.getCurrent()
        self.fileSelect(selFile)

    def fileSelect(self):
        selFile = filedialog.askopenfilename(initialdir="/", title="Select file", filetypes=(("image files", "*.jpg *.png"), ("all files", "*.*")))
        print(selFile)
        self.image = Image.open(selFile)
        print(self.image.size[0], self.image.size[1])

        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))

        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.image = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image = self.image)

    def next(self):
        selFile = pngLists.moveNext()
        self.fileSelect(selFile)
        threading.Timer(2, self.next).start()

print(sys.path)
a = Window()